# Flask + MySql 1

U ovom primeru bice prikazano povezivanje flask rest servisa i MySql baze. 

Struktura projekta:
```
flask-db1-
         |-static-
         |       |-index.html
         |       |-css
         |       |-js
         |-run.py
```


### MySql

Pristup MySql bazi iz python paketa moguće je preko nekoliko različitih connectora. Jedan od standardnih je:

```python
import mysql.connector
```

instalacija paketa
```
~$\pip install mysql-connector-python
```

Pokretanje primera:
```
~$\cd flask-db1
~$\python run.py

```

```
 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 253-788-624
```
Test:

http://localhost:8080/index.html


U formi popunite username i password i na dugme [[Sign in]] pozove se funkcija koja kreira novog korisnika u bazi.


#### Implementacija 

Konfiguracija i kreiranje objekta koji sadrži određen broj 'otvorenih' konekcija prema bazi:

In [15]:
# PART-1
import mysql.connector

dbconfig = {
    "host": 'rpi3-djordje.local',
    "database": "Agrotrail",
    "user":     "agro",
    "password": "trail"
}
pool = mysql.connector.pooling.MySQLConnectionPool(
    pool_name = "mypool", 
    pool_size = 3, **dbconfig)

Deo koda preko kojeg je moguće pročitati listu prvih 5 korisnika

In [16]:
rez = []
con = pool.get_connection()
cur = con.cursor()
cur.execute("SELECT * FROM Users LIMIT 5 OFFSET 0")
rows = cur.fetchall()
for row in rows:
    ob = {}
    for i, col in enumerate(cur.description):
        ob[col[0]] = row[i]
    rez.append(ob)
con.close()
for usr in rez:
    print usr['username']

pera
pera
pera
pera
pera


## run.py

U datoteci run.py dodacemo:

```python
import mysql.connector

dbconfig = {
    "host": 'rpi3-djordje.local',
    "database": "Agrotrail",
    "user":     "agro",
    "password": "trail"
}
pool = mysql.connector.pooling.MySQLConnectionPool(
    pool_name = "mypool", 
    pool_size = 3, **dbconfig)
```
i definisati funkcije:

In [ ]:
# get list of all users
@app.route("/api/users", methods=['GET'])
def api_users():
    rez = []
    con = pool.get_connection()
    cur = con.cursor()
    cur.execute("SELECT * FROM Users LIMIT 100 OFFSET 0")
    rows = cur.fetchall()
    for row in rows:
        ob = {}
        for i, col in enumerate(cur.description):
            ob[col[0]] = row[i]
        rez.append(ob)
    con.close()
    return jsonify(result=rez)

In [ ]:
# create users
@app.route("/api/users", methods=['POST'])
def api_users_add():
    username = request.form.get('inUsername')
    password = request.form.get('inPassword')
    con = pool.get_connection()
    cur = con.cursor(buffered=True)
    cur.execute("""
           INSERT INTO Users(Farms_idFarms, username, password)
           VALUES(1, '{0}', '{1}')""".format(username, password)
    )
    cur.execute("SELECT LAST_INSERT_ID();")
    id = cur.fetchone()
    con.close()
    return jsonify(result={
         'id':id[0], 
         'username':username, 
         'password': password})

Cela datoteka 
# run.py

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

from flask import Flask
from flask import request
from flask import jsonify
import mysql.connector

app = Flask(__name__)

@app.route('/<path:path>')
def static_file(path):
    try:    
        return app.send_static_file(path)
    except: 
        return "Error"

@app.route('/')
def root():
    return app.send_static_file('index.html')


dbconfig = {
    "host": 'rpi3-djordje.local',
    "database": "Agrotrail",
    "user":     "agro",
    "password": "trail"
}

pool = mysql.connector.pooling.MySQLConnectionPool(
    pool_name = "mypool", pool_size = 3, **dbconfig)


# get list of all users
@app.route("/api/users", methods=['GET'])
def api_users():
    rez = []
    con = pool.get_connection()
    cur = con.cursor()
    cur.execute("SELECT * FROM Users LIMIT 100 OFFSET 0")
    rows = cur.fetchall()
    for row in rows:
        ob = {}
        for i, col in enumerate(cur.description):
            ob[col[0]] = row[i]
        rez.append(ob)
    con.close()
    return jsonify(result=rez)

# create users
@app.route("/api/users", methods=['POST'])
def api_users_add():
    username = request.form.get('inUsername')
    password = request.form.get('inPassword')
    con = pool.get_connection()
    cur = con.cursor(buffered=True)
    cur.execute("""
           INSERT INTO Users(Farms_idFarms, username, password)
           VALUES(1, '{0}', '{1}')""".format(username, password)
    )
    cur.execute("SELECT LAST_INSERT_ID();")
    id = cur.fetchone()
    con.close()
    return jsonify(result={
         'id':id[0], 
         'username':username, 
         'password': password})

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=8080, debug=True)


# index.html

U datoteci index.html definisana je forma sa pozivanje funkcije:
```
@app.route("/api/users", methods=['POST'])
```

```html
  <form class="form-signin" action="/api/users" method=post>
    <h2 class="form-signin-heading">Please sign in</h2>
    <br>
    <label for="inputEmail" class="sr-only">Username</label>
    <input type="text" id="inputUsername" name="inUsername" 
           class="form-control" placeholder="Username" 
            required autofocus>
    <label for="inputPassword" class="sr-only">Password</label>
    <input type="password" id="inPassword" name="inPassword" 
           class="form-control" placeholder="Password" required>
    <button class="btn btn-lg btn-primary btn-block" 
            type="submit">
        Sign in</button>
  </form>

```
